In [40]:
import pandas as pd
import numpy as np
from os import path
from datetime import datetime
from scipy.stats import rankdata, ranksums

In [41]:
def load_pathways_genes():
    pathways = {}
    with open(pathway_file_dir, 'r') as f:
        for line in f:
            parts = line.strip().upper().split('\t')  # Split each line into parts
            if len(parts) < 3:  # If there are not enough parts for name, size, and genes
                continue

            pathway_name = parts[0]  # The pathway name is the first part
            try:
                pathway_size = int(parts[1])  # The pathway size is the second part
            except ValueError:
                continue  # Skip this line if the size is not an integer

            # Further split the gene part by spaces and then take the number of genes specified by pathway_size
            genes = parts[2]  # The third part is the space-separated list of gene IDs
            gene_list = genes.split()  # Split the genes by spaces

            # Convert each gene to an integer
            try:
                genes = [int(gene) for gene in gene_list[:pathway_size]]
            except ValueError:
                continue  # Skip this line if any gene is not an integer

            pathways[pathway_name] = genes  # Add the pathway and its genes to the dictionary

    return pathways

def get_scores():
    # Path to the file containing the raw scores (adjust as necessary)
    raw_scores_file_path = experiment_file_path

    try:
        # Load raw data from the file
        raw_data = pd.read_excel(raw_scores_file_path)

        # Perform necessary preprocessing on raw_data
        # For instance, sorting, filtering, or extracting specific columns
        # Assuming 'GeneID' and 'Score' are columns in the raw data
        sorted_raw_data = raw_data.sort_values(by='GeneID').reset_index(drop=True)

        # Create a dictionary for gene_id_to_score
        scores_dict = {gene_id: score for gene_id, score in zip(sorted_raw_data['GeneID'], sorted_raw_data['Score'])}
        return scores_dict

    except FileNotFoundError:
        print(f"File not found: {raw_scores_file_path}")
        return pd.DataFrame(), {}
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame(), {}

In [42]:
def bh_correction(p_values):
    p_vals_rank = rankdata(p_values, 'max') - 1
    p_vals_rank_ord = rankdata(p_values, 'ordinal') - 1

    p_values_sorted = np.zeros_like(p_vals_rank)
    p_values_sorted[p_vals_rank_ord] = np.arange(len(p_vals_rank_ord))

    p_vals = p_values * (len(p_values) / (p_vals_rank + 1))
    adj_p_vals_by_rank = p_vals[p_values_sorted]

    p_vals_ordered = np.minimum(adj_p_vals_by_rank, np.minimum.accumulate(adj_p_vals_by_rank[::-1])[::-1])
    adj_p_vals = p_vals_ordered[p_vals_rank]
    return adj_p_vals

In [43]:
def perform_statist():
    significant_pathways_with_genes = {}
    ks_p_values = []
    pathway_names = []
    results = {}

    # Precompute scores keys for efficiency
    scores_keys = set(scores.keys())

    # Filter genes for each pathway, only includes genes that are in the experiment and in the pathway file
    genes_by_pathway_filtered = {
        pathway: [gene_id for gene_id in genes if gene_id in scores]
        for pathway, genes in genes_by_pathway.items()
    }

    # Filter pathways based on gene count criteria
    pathways_with_many_genes = [
        pathway for pathway, genes in genes_by_pathway_filtered.items()
        if minimum_gene_per_pathway <= len(genes) <= maximum_gene_per_pathway
    ]

    print('After filtering:', len(pathways_with_many_genes))

    # Perform statistical tests
    for pathway in pathways_with_many_genes:
        filtered_genes = set(genes_by_pathway_filtered[pathway])
        pathway_scores = [scores[gene_id] for gene_id in filtered_genes]
        background_scores = [scores[gene_id] for gene_id in scores_keys - filtered_genes]

        result = statistic_test(pathway_scores, background_scores)
        results[pathway] = result
        ks_p_values.append(result[0])
        pathway_names.append(pathway)

    # Apply BH correction
    adjusted_p_values = bh_correction(np.array(ks_p_values))

    # Filter significant pathways based on adjusted p-values
    for i, pathway in enumerate(pathway_names):
        if adjusted_p_values[i] < 0.05:  # Using a significance threshold of 0.05
            significant_pathways_with_genes[pathway] = (
            genes_by_pathway_filtered[pathway], adjusted_p_values[i], results[pathway][1]
            )

    return significant_pathways_with_genes

In [44]:
def perform_statist_mann_whitney(pathways_with_many_genes):
    mw_p_values = []
    significant_pathways_with_genes = {}

    scores_keys = set(scores.keys())

    # Mann-Whitney U test and FDR
    for pathway, genes_info in pathways_with_many_genes.items():
        pathway_genes = set(genes_info[0])
        pathway_scores = [scores[gene_id] for gene_id in pathway_genes]
        background_scores = [scores[gene_id] for gene_id in scores_keys - pathway_genes]

        # Perform Mann-Whitney U Test
        mw_pval = wilcoxon_rank_sums_test(pathway_scores, background_scores)
        mw_p_values.append(mw_pval)


    # Apply BH correction to Mann-Whitney p-values
    adjusted_mw_p_values = bh_correction(np.array(mw_p_values))

    filtered_pathways = []
    for i, (pathway, genes) in enumerate(pathways_with_many_genes.items()):
        if adjusted_mw_p_values[i] < 0.05:  # Significance threshold
            significant_pathways_with_genes[pathway] = genes
            filtered_pathways.append({
                'Pathway': pathway,
                'Adjusted_p_value': adjusted_mw_p_values[i],
                'Genes': genes[0]
            })

    # Convert filtered pathways to a DataFrame
    pathways_df = pd.DataFrame(filtered_pathways)
    pathways_df.sort_values(by='Adjusted_p_value', inplace=True)

    # Filter pathways based on Jaccard index
    filtered_pathways = {}
    for i, row in pathways_df.iterrows():
        current_genes = set(row['Genes'])
        if not any(jaccard_index(current_genes, set(filtered_row['Genes'])) > JAC_THRESHOLD
                   for filtered_row in filtered_pathways.values()):
            filtered_pathways[row['Pathway']] = row

    return filtered_pathways

In [45]:
def jaccard_index(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

In [46]:
def kolmogorov_smirnov_test(experiment_scores, control_scores):
    # Convert lists to numpy arrays and sort
    experiment_scores = np.sort(experiment_scores)
    control_scores = np.sort(control_scores)
    # Initialize variables
    en1 = len(experiment_scores)
    en2 = len(control_scores)

    # Calculate empirical cumulative distribution functions for both sets
    data_all = np.concatenate([experiment_scores, control_scores])
    cdf_experiment = np.searchsorted(experiment_scores, data_all, side='right') / en1
    cdf_control = np.searchsorted(control_scores, data_all, side='right') / en2

    # Find the maximum distance
    D = np.max(np.abs(cdf_experiment - cdf_control))
    # Calculate the KS statistic
    en = np.sqrt(en1 * en2 / (en1 + en2))
    p_value = ks((en + 0.12 + 0.11 / en) * D)

    # Determine directionality
    if np.mean(experiment_scores) > 0:
        direction = 'greater'
    elif np.mean(experiment_scores) < 0:
        direction = 'less'
    else:
        direction = 'not significant'

    return p_value, direction
def kolmogorov_smirnov_test(experiment_scores, control_scores):
    # Convert lists to numpy arrays and sort
    experiment_scores = np.sort(experiment_scores)
    control_scores = np.sort(control_scores)
    # Initialize variables
    en1 = len(experiment_scores)
    en2 = len(control_scores)

    # Calculate empirical cumulative distribution functions for both sets
    data_all = np.concatenate([experiment_scores, control_scores])
    cdf_experiment = np.searchsorted(experiment_scores, data_all, side='right') / en1
    cdf_control = np.searchsorted(control_scores, data_all, side='right') / en2

    # Find the maximum distance
    D = np.max(np.abs(cdf_experiment - cdf_control))
    # Calculate the KS statistic
    en = np.sqrt(en1 * en2 / (en1 + en2))
    p_value = ks((en + 0.12 + 0.11 / en) * D)

    # Determine directionality
    if np.mean(experiment_scores) > 0:
        direction = 'greater'
    elif np.mean(experiment_scores) < 0:
        direction = 'less'
    else:
        direction = 'not significant'

    return p_value, direction


def ks(alam):
    EPS1 = 1e-6  # Convergence criterion based on the term's absolute value
    EPS2 = 1e-10  # Convergence criterion based on the sum's relative value
    a2 = -2.0 * alam ** 2  # Squared and negated lambda for exponential calculation
    fac = 2.0
    sum = 0.0
    termbf = 0.0

    # Iteratively calculate the KS probability
    for j in range(1, 101):
        term = fac * np.exp(a2 * j ** 2)  # Calculate term of the series
        sum += term  # Add to sum

        # Check for convergence
        if np.abs(term) <= EPS1 * termbf or np.abs(term) <= EPS2 * sum:
            return sum

        fac = -fac  # Alternate the sign
        termbf = np.abs(term)  # Update term before flag

    # Return 1.0 if the series does not converge in 100 terms
    return 1.0

In [47]:
def wilcoxon_rank_sums_test(experiment_scores, elements_scores, alternative='two-sided'):
    p_vals = ranksums(experiment_scores, elements_scores, alternative=alternative).pvalue
    return p_vals

In [48]:
def print_enriched_pathways(filtered_pathways, threshold=0.05):
    """
    Prints the enriched pathways that pass a specified p-value threshold.

    Args:
        filtered_pathways (dict): Dictionary containing pathways and their details.
        threshold (float): P-value threshold for filtering significant pathways.
    """
    significant_count = 0
    print("Enriched Pathways:")
    print("------------------")
    for pathway, details in filtered_pathways.items():
        p_value = details.get('Adjusted_p_value')
        if p_value and p_value < threshold:
            print(f"{pathway} P-Value: {p_value}")
            significant_count += 1

    print(f"===== {significant_count} enriched pathways\n")

In [49]:
minimum_gene_per_pathway = 20
maximum_gene_per_pathway = 200
FDR_threshold = 0.05
JAC_THRESHOLD = 0.2
root_path = '/mnt/c/Users/pickh/PycharmProjects/Yair_propagation'
output_path = '/mnt/c/Users/pickh/PycharmProjects/Yair_propagation/Outputs'
figure_name = 'figure'
figure_title = 'Pathway Enrichment'
experiment_name = "roded_T_v_N"
species = 'H_sapiens'
data_file = 'Data'
genes_names_file = 'H_sapiens.gene_info'  # optional if needed
date = datetime.today().strftime('%d_%m_%Y__%H_%M_%S')
pathway_file = 'pathways'
data_dir = path.join(root_path, data_file)
genes_names_file_path = path.join(data_dir, species, 'genes_names', genes_names_file)
pathway_file_dir = path.join(data_dir, species, 'pathways', pathway_file)
input_dir = path.join(root_path, 'Inputs', 'experiments_data', 'Parkinson')
experiment_file_path = path.join(input_dir, f'{experiment_name}.xlsx')
output_folder = path.join(root_path, 'Outputs', 'enrichment_scores', experiment_name)
statistic_test = kolmogorov_smirnov_test
results = dict()
genes_by_pathway = load_pathways_genes()
scores = get_scores()

In [50]:
print("running enrichment")
significant_pathways_with_genes = perform_statist()

running enrichment
After filtering: 1059


In [51]:
filtered_pathways = perform_statist_mann_whitney(significant_pathways_with_genes)

In [52]:
print("finished enrichment")
print_enriched_pathways(filtered_pathways, FDR_threshold)

finished enrichment
Enriched Pathways:
------------------
REACTOME_SCF_SKP2_MEDIATED_DEGRADATION_OF_P27_P21 P-Value: 1.82477769417367e-08
REACTOME_ACTIVATION_OF_THE_MRNA_UPON_BINDING_OF_THE_CAP_BINDING_COMPLEX_AND_EIFS_AND_SUBSEQUENT_BINDING_TO_43S P-Value: 1.82477769417367e-08
WP_T_CELL_MODULATION_IN_PANCREATIC_CANCER P-Value: 2.18773019136929e-06
WP_COMPLEMENT_SYSTEM P-Value: 4.2961000375710236e-05
WP_PHOTODYNAMIC_THERAPYINDUCED_HIF1_SURVIVAL_SIGNALING P-Value: 8.59732062341705e-05
REACTOME_ELASTIC_FIBRE_FORMATION P-Value: 0.00010622948158491842
REACTOME_PERK_REGULATES_GENE_EXPRESSION P-Value: 0.00013926679579700103
WP_OXIDATIVE_PHOSPHORYLATION P-Value: 0.00014417742347176674
WP_DNA_REPLICATION P-Value: 0.0001508222462499153
REACTOME_MITOCHONDRIAL_FATTY_ACID_BETA_OXIDATION P-Value: 0.00016679440693938405
WP_ALLOGRAFT_REJECTION P-Value: 0.00019551341716243908
WP_RETT_SYNDROME_CAUSING_GENES P-Value: 0.00019551341716243908
REACTOME_RUNX1_REGULATES_GENES_INVOLVED_IN_MEGAKARYOCYTE_DIFFERE